# Prepare


In [1]:
import polars as pl
import re
from tqdm.notebook import tqdm

DATA_PREFIX = "./data"
RAW_DATA_PREFIX = f"{DATA_PREFIX}/raw"
INDEX_EDGES_PREFIX = f"{DATA_PREFIX}/index"

import os

if not os.path.exists(INDEX_EDGES_PREFIX):
    os.makedirs(INDEX_EDGES_PREFIX)
if not os.path.exists(RAW_DATA_PREFIX):
    os.makedirs(RAW_DATA_PREFIX)


def get_inner_namespace(col_name: str) -> str:
    match = re.search("\((.*?)\)", col_name)
    return "" if match is None else match.group(1)


def get_namespace(col_name: str) -> str:
    inner_namespace = get_inner_namespace(col_name)
    if inner_namespace in ["Country", "Continent", "City"]:
        return "Placeid"
    if inner_namespace in ["University", "Company"]:
        return "Organisationid"
    return inner_namespace


# test
get_namespace(":ID(Forumid)")

'Forumid'

In [2]:
""" Test """

PLACE = f"{RAW_DATA_PREFIX}/place.csv"

df = pl.read_csv(PLACE)
test_table = df.lazy().filter(pl.col("name").is_in(["India", "China"])).collect()
out = test_table.select(
    [
        pl.col(":ID(Placeid)"),
        pl.col("name"),
        pl.col(":TYPE"),
        pl.col(":LABEL"),
    ]
)
out.head(5)

:ID(Placeid),name,:TYPE,:LABEL
i64,str,str,str
0,"""India""","""country""","""place"""
1,"""China""","""country""","""place"""


In [3]:
""" Load `vertices/edges` """

import os, glob
from polars import DataFrame

vertices = dict[str, DataFrame]()
raw_edges = dict[tuple[str, str, str], DataFrame]()
switch_namespace = dict[str, dict[int, int]]()

for file in glob.glob(f"{RAW_DATA_PREFIX}/*.csv"):
    df_name = os.path.basename(file).split(".")[0]
    if "_" in df_name:
        src, relationship, dst = df_name.split("_")
        raw_edges[(src, relationship, dst)] = pl.read_csv(file)
    else:
        vertices[df_name] = pl.read_csv(file)


vertex_num = sum(len(df) for df in vertices.values())
edge_num = sum(len(df) for df in raw_edges.values())

In [4]:
""" Initialize `switch_namespace` """

for df in vertices.values():
    namespace = get_namespace(df.columns[0])
    switch_namespace[namespace] = dict()

switch_namespace

{'Commentid': {},
 'Forumid': {},
 'Organisationid': {},
 'Personid': {},
 'Placeid': {},
 'Postid': {},
 'Tagid': {},
 'TagClassid': {}}

In [5]:
""" Re-arrange all `:ID($AnyNamespace)` """

curr_global_id = 0

with tqdm(desc="Mapping `origin_id` to `uni_id`", total=vertex_num) as bar:
    for df in vertices.values():
        namespace = get_namespace(df.columns[0])
        map = switch_namespace[namespace]
        for row in df.rows():
            map[int(row[0])] = curr_global_id
            curr_global_id += 1
            bar.update(1)

assert curr_global_id == vertex_num
switch_namespace.keys()

Mapping `origin_id` to `uni_id`:   0%|          | 0/3181724 [00:00<?, ?it/s]

dict_keys(['Commentid', 'Forumid', 'Organisationid', 'Personid', 'Placeid', 'Postid', 'Tagid', 'TagClassid'])

# Original Query


In [6]:
OUT_PREFIX = "./out"
ORIGINAL_QUERY_PREFIX = f"{OUT_PREFIX}/original"
DATA_GRAPH = f"{ORIGINAL_QUERY_PREFIX}/data_graph.txt"
CHINA_QUERY_GRAPH = f"{ORIGINAL_QUERY_PREFIX}/china_query_graph.txt"
INDIA_QUERY_GRAPH = f"{ORIGINAL_QUERY_PREFIX}/india_query_graph.txt"

import os

if not os.path.exists(ORIGINAL_QUERY_PREFIX):
    os.makedirs(ORIGINAL_QUERY_PREFIX)

In [7]:
""" Build map of `vertex.uni_id -> label` """

labels = dict[int, str]()
label_set = set[str]()


def place_op(df: DataFrame):
    namespace = get_namespace(df.columns[0])
    map = switch_namespace[namespace]
    slice = df.select(
        [
            pl.col(df.columns[0]),
            pl.col("name"),
            pl.col(":TYPE"),
        ]
    )
    for origin_id, name, ty in slice.rows():
        name, label = str(name), str(ty)
        uni_id = map[int(origin_id)]
        if name in ["China", "India"]:
            label = name
        labels[uni_id] = label
        label_set.add(label)
        bar.update(1)


def normal_op(df: DataFrame):
    namespace = get_namespace(df.columns[0])
    map = switch_namespace[namespace]
    slice = df.select(
        [
            pl.col(df.columns[0]),
            pl.col(":TYPE" if ":TYPE" in df.columns else ":LABEL"),
        ]
    )
    for origin_id, label in slice.rows():
        uni_id = map[int(origin_id)]
        labels[uni_id] = str(label)
        label_set.add(str(label))
        bar.update(1)


def vertex_op(df_name: str, df: DataFrame):
    (place_op if df_name == "place" else normal_op)(df)


with tqdm(desc="Build map of `vertex.uni_id -> label`", total=vertex_num) as bar:
    for df_name, df in vertices.items():
        vertex_op(df_name, df)

label_set

Build map of `vertex.uni_id -> label`:   0%|          | 0/3181724 [00:00<?, ?it/s]

{'China',
 'India',
 'city',
 'comment',
 'company',
 'continent',
 'country',
 'forum',
 'person',
 'post',
 'tag',
 'tagclass',
 'university'}

In [8]:
""" Build edges in format: `(src_id, dst_id)` """

edges = set[tuple[int, int]]()

with tqdm(desc="Build edges in format: `(src_id, dst_id)`", total=edge_num) as bar:
    for df_name, df in raw_edges.items():
        src_namespace = get_namespace(df.columns[0])
        dst_namespace = get_namespace(df.columns[1])
        src_map = switch_namespace[src_namespace]
        dst_map = switch_namespace[dst_namespace]
        slice = df.select(
            [
                pl.col(df.columns[0]),
                pl.col(df.columns[1]),
            ]
        )
        for src_id, dst_id in slice.rows():
            src_uni_id = src_map[int(src_id)]
            dst_uni_id = dst_map[int(dst_id)]
            edges.add((src_uni_id, dst_uni_id))
            bar.update(1)

Build edges in format: `(src_id, dst_id)`:   0%|          | 0/17256038 [00:00<?, ?it/s]

In [9]:
""" Write into `data_graph.txt` """

if not os.path.exists(DATA_GRAPH):
    with open(DATA_GRAPH, "w") as f:
        f.write("#0\n")
        f.write(f"{len(labels)}\n")
        with tqdm(
            desc="Writing `labels` into `data_graph.txt`", total=len(labels)
        ) as bar:
            for i in range(len(labels)):
                f.write(f"{labels[i]}\n")
                bar.update(1)
        f.write(f"{len(edges)}\n")
        with tqdm(desc="Writing `edges` into `data_graph.txt", total=len(edges)) as bar:
            for src, dst in edges:
                f.write(f"{src} {dst}\n")
                bar.update(1)
else:
    print(f"File `{DATA_GRAPH}` already exists")

File `./out/original/data_graph.txt` already exists


In [10]:
""" Build `India` and `China` query graph """

china_query_graph_labels = ["China"] + ["city"] * 3 + ["person"] * 3
china_query_graph_edges = [
    (1, 0),
    (2, 0),
    (3, 0),
    (4, 1),
    (5, 2),
    (6, 3),
    (4, 5),
    (5, 6),
    (6, 4),
]

india_query_graph_labels = ["India"] + ["city"] * 3 + ["person"] * 3
india_query_graph_edges = china_query_graph_edges

if not os.path.exists(CHINA_QUERY_GRAPH):
    with open(CHINA_QUERY_GRAPH, "w") as f:
        f.write("#0\n")
        f.write(f"{len(china_query_graph_labels)}\n")
        [f.write(f"{label}\n") for label in china_query_graph_labels]
        f.write(f"{len(china_query_graph_edges)}\n")
        [f.write(f"{src} {dst}\n") for src, dst in china_query_graph_edges]

if not os.path.exists(INDIA_QUERY_GRAPH):
    with open(INDIA_QUERY_GRAPH, "w") as f:
        f.write("#0\n")
        f.write(f"{len(india_query_graph_labels)}\n")
        [f.write(f"{label}\n") for label in india_query_graph_labels]
        f.write(f"{len(india_query_graph_edges)}\n")
        [f.write(f"{src} {dst}\n") for src, dst in india_query_graph_edges]

# Optimized Query


In [11]:
OPTIMIZED_QUERY_PREFIX = f"{OUT_PREFIX}/optimized"
DATA_GRAPH = f"{OPTIMIZED_QUERY_PREFIX}/data_graph.txt"
CHINA_QUERY_GRAPH = f"{OPTIMIZED_QUERY_PREFIX}/china_query_graph.txt"
INDIA_QUERY_GRAPH = f"{OPTIMIZED_QUERY_PREFIX}/india_query_graph.txt"

if not os.path.exists(OPTIMIZED_QUERY_PREFIX):
    os.makedirs(OPTIMIZED_QUERY_PREFIX)

In [12]:
""" Load all `index edge` """

index_edges = dict[str, DataFrame]()

for file in glob.glob(f"{INDEX_EDGES_PREFIX}/*.csv"):
    df_name = os.path.basename(file).split(".")[0]
    index_edges[df_name] = pl.read_csv(file)

index_edge_num = sum(len(df) for df in index_edges.values())

In [13]:
""" Add `index edge` into `edges` """

with tqdm(desc="Adding `index edge` into `edges`", total=index_edge_num) as bar:
    for df in index_edges.values():
        src_namespace = get_namespace(df.columns[0])
        dst_namespace = get_namespace(df.columns[1])
        src_map = switch_namespace[src_namespace]
        dst_map = switch_namespace[dst_namespace]
        slice = df.select(
            [
                pl.col(df.columns[0]),
                pl.col(df.columns[1]),
            ]
        )
        for src_id, dst_id in slice.rows():
            src_uni_id = src_map[int(src_id)]
            dst_uni_id = dst_map[int(dst_id)]
            edges.add((src_uni_id, dst_uni_id))
            bar.update(1)

Adding `index edge` into `edges`:   0%|          | 0/19593582 [00:00<?, ?it/s]

In [14]:
""" Write into `data_graph.txt` """

if not os.path.exists(DATA_GRAPH):
    with open(DATA_GRAPH, "w") as f:
        f.write("#0\n")
        f.write(f"{len(labels)}\n")
        with tqdm(
            desc="Writing `labels` into `data_graph.txt`", total=len(labels)
        ) as bar:
            for i in range(len(labels)):
                f.write(f"{labels[i]}\n")
                bar.update(1)
        f.write(f"{len(edges)}\n")
        with tqdm(desc="Writing `edges` into `data_graph.txt", total=len(edges)) as bar:
            for src, dst in edges:
                f.write(f"{src} {dst}\n")
                bar.update(1)
else:
    print(f"File `{DATA_GRAPH}` already exists")

File `./out/optimized/data_graph.txt` already exists


In [15]:
""" Build `India` and `China` query graph """

china_query_graph_labels = ["China"] + ["person"] * 3
china_query_graph_edges = [
    (1, 0),
    (2, 0),
    (3, 0),
    (1, 2),
    (2, 3),
    (3, 1),
]

india_query_graph_labels = ["India"] + ["person"] * 3
india_query_graph_edges = china_query_graph_edges

if not os.path.exists(CHINA_QUERY_GRAPH):
    with open(CHINA_QUERY_GRAPH, "w") as f:
        f.write("#0\n")
        f.write(f"{len(china_query_graph_labels)}\n")
        [f.write(f"{label}\n") for label in china_query_graph_labels]
        f.write(f"{len(china_query_graph_edges)}\n")
        [f.write(f"{src} {dst}\n") for src, dst in china_query_graph_edges]

if not os.path.exists(INDIA_QUERY_GRAPH):
    with open(INDIA_QUERY_GRAPH, "w") as f:
        f.write("#0\n")
        f.write(f"{len(india_query_graph_labels)}\n")
        [f.write(f"{label}\n") for label in india_query_graph_labels]
        f.write(f"{len(india_query_graph_edges)}\n")
        [f.write(f"{src} {dst}\n") for src, dst in india_query_graph_edges]

# Execute `Query`


In [18]:
import subprocess
import platform


ORIGINAL_DATA_GRAPH = f"{ORIGINAL_QUERY_PREFIX}/data_graph.txt"
ORIGINAL_CHINA_QUERY_GRAPH = f"{ORIGINAL_QUERY_PREFIX}/china_query_graph.txt"
ORIGINAL_INDIA_QUERY_GRAPH = f"{ORIGINAL_QUERY_PREFIX}/india_query_graph.txt"

OPTIMIZED_DATA_GRAPH = f"{OPTIMIZED_QUERY_PREFIX}/data_graph.txt"
OPTIMIZED_CHINA_QUERY_GRAPH = f"{OPTIMIZED_QUERY_PREFIX}/china_query_graph.txt"
OPTIMIZED_INDIA_QUERY_GRAPH = f"{OPTIMIZED_QUERY_PREFIX}/india_query_graph.txt"


LOG_PREFIX = f"./log"
ORIGINAL_LOG_PREFIX = f"{LOG_PREFIX}/original"
OPTIMIZED_LOG_PREFIX = f"{LOG_PREFIX}/optimized"

if not os.path.exists(ORIGINAL_LOG_PREFIX):
    os.makedirs(ORIGINAL_LOG_PREFIX)
if not os.path.exists(OPTIMIZED_LOG_PREFIX):
    os.makedirs(OPTIMIZED_LOG_PREFIX)

ORIGINAL_CHINA_MATCH_RESULT = f"{ORIGINAL_LOG_PREFIX}/china_match_result.txt"
ORIGINAL_INDIA_MATCH_RESULT = f"{ORIGINAL_LOG_PREFIX}/india_match_result.txt"

OPTIMIZED_CHINA_MATCH_RESULT = f"{OPTIMIZED_LOG_PREFIX}/china_match_result.txt"
OPTIMIZED_INDIA_MATCH_RESULT = f"{OPTIMIZED_LOG_PREFIX}/india_match_result.txt"

wsl_if_on_windows = ["wsl"] if platform.system() == "Windows" else []

# ./VEQ_M_100k -dg <data_graph_path> -qg <query_graph_path>

original_china_match_args = wsl_if_on_windows + [
    "./VEQ_M_100k",
    "-dg",
    ORIGINAL_DATA_GRAPH,
    "-qg",
    ORIGINAL_CHINA_QUERY_GRAPH,
]
original_india_match_args = wsl_if_on_windows + [
    "./VEQ_M_100k",
    "-dg",
    ORIGINAL_DATA_GRAPH,
    "-qg",
    ORIGINAL_INDIA_QUERY_GRAPH,
]

# ./VEQ_M_100k -dg <data_graph_path> -qg <query_graph_path>

optimized_china_match_args = wsl_if_on_windows + [
    "./VEQ_M_100k",
    "-dg",
    OPTIMIZED_DATA_GRAPH,
    "-qg",
    OPTIMIZED_CHINA_QUERY_GRAPH,
]
optimized_india_match_args = wsl_if_on_windows + [
    "./VEQ_M_100k",
    "-dg",
    OPTIMIZED_DATA_GRAPH,
    "-qg",
    OPTIMIZED_INDIA_QUERY_GRAPH,
]


def run(result_path: str, task_name: str, args: list[str]):
    with open(result_path, "w") as f:
        print(f">>> Running: {task_name}...")
        p = subprocess.Popen(
            args,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        if p.stdout:
            for line in iter(p.stdout.readline, b""):
                content = line.decode("utf-8")
                print("    " + content, end="")
                f.write(content)
        else:
            print("    <No output>")
            f.write("<No output>")
        print("<<< Done!")

In [19]:
""" Exec `match` on `original` """

run(ORIGINAL_CHINA_MATCH_RESULT, "original_china_match", original_china_match_args)
run(ORIGINAL_INDIA_MATCH_RESULT, "original_india_match", original_india_match_args)

""" Exec `match` on `optimized` """

run(OPTIMIZED_CHINA_MATCH_RESULT, "optimized_china_match", optimized_china_match_args)
run(OPTIMIZED_INDIA_MATCH_RESULT, "optimized_india_match", optimized_india_match_args)

>>> Running: original_china_match...
    Data file: ./out/original/data_graph.txt
    Query file: ./out/original/china_query_graph.txt
    Output file: 
    Sum of |C(u)|: 3748
    Total Recursive Call Count: 131167
    Number of Matches: 100000
    Filtering Time (ms): 23.7614
    Verification Time (ms): 5987.59
    Processing Time (ms): 6011.35
<<< Done!
>>> Running: original_india_match...
    Data file: ./out/original/data_graph.txt
    Query file: ./out/original/china_query_graph.txt
    Output file: 
    Sum of |C(u)|: 3748
    Total Recursive Call Count: 131167
    Number of Matches: 100000
    Filtering Time (ms): 19.31
    Verification Time (ms): 6918.57
    Processing Time (ms): 6937.88
<<< Done!
>>> Running: optimized_china_match...
    Data file: ./out/original/data_graph.txt
    Query file: ./out/original/china_query_graph.txt
    Output file: 
    Sum of |C(u)|: 3748
    Total Recursive Call Count: 131167
    Number of Matches: 100000
    Filtering Time (ms): 19.5624
    